Simple program for special quasirandom structure (SQS) derivation (Phys. Rev. Lett. **65**, 353 (1990)). Take $P6/mmm$ lattice as example. For high-precision algorithm one may refer to (Phys. Rev. Materials **5**, 113803 (2021)).

<figure>
    <img src="utils/cij_1100_bulk.png" width="400px"/>
    <figcaption>Randomization target is the atomic arrangement of transition metal elements.</figcaption>
</figure>

In [1]:
using Random
using LinearAlgebra
using NPZ
using Combinatorics
using Statistics
using JLD2
using Distributed
using JSON
using Base.Threads
using IJulia
using Profile
using BenchmarkTools
using .GC
using PyPlot

function zero_clip(num, x)
    if x == 0
        return 0
    end
    
    return num
end

println(Threads.nthreads())
pth_loadbase = ""
@everywhere include("$(pth_loadbase)cpr_rspace.jl")
@everywhere include("$(pth_loadbase)basis_func_quin.jl")
@everywhere include("$(pth_loadbase)cpr_sqs.jl")

8


In [ ]:
atom_num = 64 #TODO number of atoms
pth_loadbase = ""
basis_norm_dict_pth = "$(pth_loadbase)utils/basis_norm_dict_jl_quin.json" 

ind_cluster_dict_pth = "$(pth_loadbase)utils/1100/tm_4_4_2_64/cluster_ind_4_4_2_jl.json"
basis_norm_dict = JSON.parsefile(basis_norm_dict_pth)
ind_cluster_dict = JSON.parsefile(ind_cluster_dict_pth)

symbol_list_pth = "$(pth_loadbase)utils/symbol_list_240111_6nn.json"
symbol_list = JSON.parsefile(symbol_list_pth)
println(length(symbol_list))

config_info = config(
        ind_cluster_dict,
        symbol_list,
        basis_norm_dict,
        # cluster_norm_dict,
        true, true, true, true)

Main part

In [ ]:
function spin_exchange(sigma, action)
    sigma_n = copy(sigma)
    sigma_n[action[1]], sigma_n[action[2]] = sigma[action[2]], sigma[action[1]]
    return sigma_n
end

compo_list = Matrix{Float64}([
    0 0.3 0.4 0.3 0;
]) #* composition

compo_denote = join(convert(Vector{Int64}, round.(compo_list[1,:]*100),), "")
#TODO Set the path
system_denote = "ter_Cij"
sav_pth = "sqs_zoo"
meta_iter = 12
atom_num = 64 #TODO number of transition metal atom
lagrange_list = LinRange(1e-5, 1e-3, meta_iter)

# Threads.@threads for meta_i in 1:meta_iter
for meta_i in 1:meta_iter

    #TODO specify the MC steps
    mc_iter = 10000
    lagrange = lagrange_list[meta_i]
    # lagrange = 1e-5
    temp = 2
    res_min = Inf
    ele_list_min, cprvec_min = nothing, nothing
    res_list = []
    global res_list
    cprvec_sqs = main_sqs(config_info, compo_list)[1] #TODO as the ground truth

    #TODO rule for sequence generation
    ele_list = ele_list_gen(compo_list[1,:], atom_num, 1, "randchoice")
    cpr_sigma = main_rspace(1:1, config_info, ele_list)[1]
    res_raw = norm(cpr_sigma.-cprvec_sqs)

    for i in 1:mc_iter 
        temp = exp(-i/100)
        action = rand(1:atom_num, 2)
        ele_list_n = spin_exchange(ele_list, action)
        cpr_sigma_n = main_rspace(1:1, config_info, ele_list_n)[1]
        res_n = norm(cpr_sigma_n.-cprvec_sqs)
        penalize = maximum(abs.(cpr_sigma_n.-cprvec_sqs))*lagrange
        res_n += penalize 
        append!(res_list, res_n)

        if minimum([1, exp((res_raw - res_n)/temp)]) > rand()
            ele_list = copy(ele_list_n)
            res_raw = copy(res_n)
            if res_n < res_min
                res_min = copy(res_n)
                cprvec_min = copy(cpr_sigma_n)
                ele_list_min = copy(ele_list)
            end
        end
    end
    npzwrite(joinpath(sav_pth, "sqs_$(system_denote)_$(meta_i)_$(atom_num)_$(compo_denote).npy"), ele_list_min)
end